In [ ]:
%pip install --user pdfplumber

%pip install --user ipywidgets pymupdf

%pip install --user ipywidgets

In [ ]:
%pip install --user reportlab

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pdfplumber
import fitz  # PyMuPDF
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

def extract_data_using_pdfplumber(file_path):
    with pdfplumber.open(file_path) as pdf:
        first_page = pdf.pages[0]
        text = first_page.extract_text(x_tolerance=3, y_tolerance=3)  # Modify tolerance as needed
        
        # For example, if you know checkbox coordinates
        checkboxes = [{
            'bbox': (100, 200, 110, 210),  # Example coordinates
            'label': 'Checkbox 1'
        }]
        extracted_data = []
        for checkbox in checkboxes:
            # Get words near the checkbox, this needs precise coordinates
            words = first_page.extract_words()
            for word in words:
                if word['x0'] >= checkbox['bbox'][0] and word['x1'] <= checkbox['bbox'][2] and word['top'] >= checkbox['bbox'][1] and word['bottom'] <= checkbox['bbox'][3]:
                    extracted_data.append((checkbox['label'], 'Checked'))  # Simulated check state
        return text, extracted_data

def create_summary_document(output_path, extracted_data):
    c = canvas.Canvas(output_path, pagesize=letter)
    text_object = c.beginText(40, 750)
    text_object.setFont("Helvetica", 12)
    for data in extracted_data:
        text_object.textLine(f"{data[0]}: {data[1]}")
    c.drawText(text_object)
    c.save()

def on_file_upload(change):
    uploaded_files = change.new
    if uploaded_files:
        for file_info in uploaded_files.values():
            file_path = "/tmp/" + file_info['metadata']['name']
            with open(file_path, 'wb') as f:
                f.write(file_info['content'])
            extracted_text, extracted_data = extract_data_using_pdfplumber(file_path)
            print("Extracted Text:")
            print(extracted_text)
            print("Extracted Data:")
            for data in extracted_data:
                print(data)
            # Create a summary document with the extracted data
            create_summary_document("summary_output.pdf", extracted_data)

upload_button = widgets.FileUpload(
    accept='.pdf',
    multiple=False
)
upload_button.observe(on_file_upload, names='data')
display(upload_button)
